#### EX 4.1 FewShotPromptTemplate

기본적으로 Fewshot은 모델에게 예제들을 준다는 뜻으로
더 나은 대답을 할 수 있도록 유도 하는 예제들을 더 줄 수있다.

예를 들어 고객센터 등에서 고객응대를 할 때 어떤 식으로 응대 할지를 형식화 시켜줄수 있다.

예제들을 리스트로 만들어서 적용시켜보자

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

# 첫번쨰 단계 - 예제의 형식지정
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

# 직접 질의하는 경우
# chat.predict("What do you know about France?")

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: Franch
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Cheese
        Currency: Euro
        """,
    },
]

# 템플릿 기본 형식
# example_template = """
#    Human: {question}
#    AI: {answer}
# """

example_prompt = PromptTemplate.from_template("Human:{question}\nAI:{answer}")

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    # 여기에 질문을 넣어주자
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

# prompt.format(country="Germany")

chain = prompt | chat

chain.invoke({"country": "Korea"})

AI:
        I know this:
        Capital: Seoul
        Language: Korean
        Food: Kimchi and Bibimbap
        Currency: South Korean Won

AIMessageChunk(content='AI:\n        I know this:\n        Capital: Seoul\n        Language: Korean\n        Food: Kimchi and Bibimbap\n        Currency: South Korean Won')

##### Ex 채팅봇에게 적용하기 위한 코드 수정
- import 부분
> FewShotPromptTemplate > FewShotChatMessagePromptTemplate 로 변경  
> PromptTemplate > ChatPromptTemplate 로 변경

결과 : 조금이나마 코드를 간결하게 정리할수 있다.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {country}?"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert, you give short answers."),
        example_prompt,
        ("human", "What do you know about {country}?"),
    ]
)

chain = final_prompt | chat

chain.invoke({"country": "Thailand"})

### Example_selector 적용
Example을 선택해서 답변을 할수 있도록 example_selector를 적용해보자.   
examples의 max_length를 정해서 특정한 길이만큼의 example만 참조하도록 할수 있다. (비용절약)


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector import LengthBasedExampleSelector

# 첫번쨰 단계 - 예제의 형식지정
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

# 직접 질의하는 경우
# chat.predict("What do you know about France?")

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: Franch
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Cheese
        Currency: Euro
        """,
    },
]

# 템플릿 기본 형식
# example_template = """
#    Human: {question}
#    AI: {answer}
# """

example_prompt = PromptTemplate.from_template("Human:{question}\nAI:{answer}")

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=80,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    # examples=examples,
    example_selector=example_selector,
    # 여기에 질문을 넣어주자
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

# prompt.format(country="Germany")

chain = prompt | chat

chain.invoke({"country": "Korea"})

### Random Example selector 적용  
random으로 Example을 선택해서 적용할수 있도록 해보자. 

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector.base import BaseExampleSelector

# 첫번쨰 단계 - 예제의 형식지정
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

# 직접 질의하는 경우
# chat.predict("What do you know about France?")

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: Franch
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Cheese
        Currency: Euro
        """,
    },
]


class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]


# 템플릿 기본 형식
# example_template = """
#    Human: {question}
#    AI: {answer}
# """

example_prompt = PromptTemplate.from_template("Human:{question}\nAI:{answer}")

example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    # examples=examples,
    example_selector=example_selector,
    # 여기에 질문을 넣어주자
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

# prompt.format(country="Germany")

chain = prompt | chat

chain.invoke({"country": "Korea"})

AI:
        Here is what I know:
        Capital: Seoul
        Language: Korean
        Food: Kimchi, Bibimbap, Bulgogi
        Currency: South Korean Won

AIMessageChunk(content='AI:\n        Here is what I know:\n        Capital: Seoul\n        Language: Korean\n        Food: Kimchi, Bibimbap, Bulgogi\n        Currency: South Korean Won')